Movie Review Analysis

In [ ]:
%tensorflow_version 2.x 
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)
#movie data set, data is already preprocessed and labelled with integers. integer number is essentially a rank of frequency
#integer 0 is the most common word

Using TensorFlow backend.


17465344/17464789 [==============================] - 1s 0us/step


In [ ]:
#cannot pass data into NN that is of differing lengths, so need more preprocessing
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

#if the review is greater than 250 words then trim off the extra words
#if the review is less than 250 words add the necessary amount of 0's to make it equal to 250.

In [ ]:
# create the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),  # transform the words into vectors, of 32 dimensions
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")                           
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          2834688   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# compile and training the model
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 15s 24ms/step - loss: 0.4198 - acc: 0.8081 - val_loss: 0.2882 - val_acc: 0.8880
Epoch 2/10
625/625 [==============================] - 14s 23ms/step - loss: 0.2455 - acc: 0.9093 - val_loss: 0.3277 - val_acc: 0.8730
Epoch 3/10
625/625 [==============================] - 14s 23ms/step - loss: 0.1885 - acc: 0.9311 - val_loss: 0.2811 - val_acc: 0.8852
Epoch 4/10
625/625 [==============================] - 15s 23ms/step - loss: 0.1585 - acc: 0.9423 - val_loss: 0.2967 - val_acc: 0.8892
Epoch 5/10
625/625 [==============================] - 15s 23ms/step - loss: 0.1308 - acc: 0.9539 - val_loss: 0.3230 - val_acc: 0.8830
Epoch 6/10
625/625 [==============================] - 14s 23ms/step - loss: 0.1115 - acc: 0.9613 - val_loss: 0.3071 - val_acc: 0.8924
Epoch 7/10
625/625 [==============================] - 14s 23ms/step - loss: 0.0974 - acc: 0.9668 - val_loss: 0.3661 - val_acc: 0.8792
Epoch 8/10
625/625 [==============================] - 14s 23ms

In [ ]:
# evaluate model
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 5s 6ms/step - loss: 0.4189 - acc: 0.8538
[0.41894862055778503, 0.8537999987602234]


In [ ]:
# making predictions
# change new text into the encoded preprocessed data from the dataset
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

In [ ]:
# decode data from integer to words

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [ ]:
# make a prediction
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was awesome! Really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)


[0.28831795]
[0.24100985]
